<a href="https://colab.research.google.com/github/SemihDemir13/chatbot-project-data/blob/main/V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
print("Gerekli kütüphaneler kuruluyor...")
!pip install -q -U transformers huggingface_hub accelerate sentence-transformers torch langchain langchain-community langchain-huggingface chromadb pandas
print("Kurulum tamamlandı.")

In [ ]:
#2
from huggingface_hub import login

# Token'ını girdikten sonra 'Login successful' yazısını görmelisin.
login()

In [ ]:
#3
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("Model yükleniyor... (Biraz sabır)")

model_id = "google/gemma-2b-it"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # Colab GPU'su kullanılsın diye device_map="auto" diyoruz
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto"
    )

    # Text generation pipeline'ı oluşturuyoruz
    llm_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=250 # Cevabın uzunluğu
    )
    print("✅ Model başarıyla yüklendi!")

except Exception as e:
    print(f"❌ HATA: Model yüklenemedi. Lütfen HÜCRE 2'de token girdiğinizden ve modelin lisansını kabul ettiğinizden emin olun.\nHata: {e}")


In [ ]:
#4
import pandas as pd
import sys

print("CSV dosyası okunuyor ve veri formatlanıyor...")

try:
    df = pd.read_csv('netflix_titles.csv')
    df = df.fillna('Bilinmiyor')

    dokumanlar = []

    # Tüm veriyi alıyoruz
    print(f"Toplam {len(df)} adet kayıt işleniyor...")

    for index, row in df.iterrows():
        # Formatı sadeleştirdik. Başlık en üstte.
        metin = (
            f"Başlık: {row['title']}\n"
            f"Yönetmen: {row['director']}\n"
            f"Yıl: {row['release_year']}\n"
            f"Tür: {row['listed_in']}\n"
            f"Özet: {row['description']}\n"
            f"Detay: Bu film {row['release_year']} yılında yayınlanmıştır. {row['type']} kategorisindedir."
        )
        dokumanlar.append(metin)

    print(f"✅ {len(dokumanlar)} adet içerik başarıyla hazırlandı.")

except FileNotFoundError:
    print("❌ HATA: Dosya yok.")

In [ ]:
#5
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

print("Vektör veritabanı (Çok Dilli) oluşturuluyor...")

# DEĞİŞİKLİK BURADA:
# Artık Türkçe-İngilizce eşleştirmesi yapabilen 'multilingual' modeli kullanıyoruz.
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# ChromaDB veritabanını kur
# Eğer hata alırsan önceki veritabanını temizlemek gerekebilir ama genellikle üzerine yazar.
vektor_veritabani = Chroma.from_texts(dokumanlar, embedding=embedding_model)

print("✅ Çok dilli veritabanı hazır! Artık Türkçe sorguları İngilizce verilerle eşleştirebilirim.")

In [ ]:
#6
import re
import random

def rag_yanit(soru, top_k=3):
    # 1. ADIM: Soruda YIL geçiyor mu? (Regex ile 4 haneli sayı kontrolü)
    yil_tespiti = re.search(r'(\d{4})', soru)

    baglam = ""
    kaynak_yontem = ""

    # --- SENARYO A: YIL VARSA (Pandas Filtresi) ---
    if yil_tespiti:
        aranan_yil = int(yil_tespiti.group(1))
        kaynak_yontem = "DOĞRUDAN VERİ FİLTRESİ (PANDAS)"

        # DataFrame üzerinden filtrele
        uygun_filmler = df[df['release_year'] == aranan_yil]

        if not uygun_filmler.empty:
            secilenler = uygun_filmler.sample(n=min(2, len(uygun_filmler)))
            context_list = []

            print(f"\n[SİSTEM: {aranan_yil} yılı için {len(uygun_filmler)} film bulundu. Listeleniyor...]")

            for index, row in secilenler.iterrows():
                metin = (
                    f"Başlık: {row['title']}\n"
                    f"Yönetmen: {row['director']}\n"
                    f"Yıl: {row['release_year']}\n"
                    f"Tür: {row['listed_in']}\n"
                    f"Özet: {row['description']}"
                )
                print(f"✅ {row['title']} ({row['listed_in']})")
                context_list.append(metin)

            baglam = "\n###\n".join(context_list)
        else:
            return f"Veritabanımızda {aranan_yil} yapımı film bulunamadı."

    # --- SENARYO B: YIL YOKSA (Yapay Zeka Araması) ---
    else:
        kaynak_yontem = "VEKTÖR ARAMASI (AI)"
        aranan_dokumanlar = vektor_veritabani.similarity_search(soru, k=top_k)

        print(f"\n[SİSTEM: Konuya göre AI araması yapılıyor...]")
        context_list = []
        for doc in aranan_dokumanlar:
            # Başlığı satır satır arayıp bulalım (Hata riskini sıfırlar)
            lines = doc.page_content.split('\n')
            baslik = "Bilinmiyor"
            for satir in lines:
                if "Başlık:" in satir:
                    baslik = satir.replace("Başlık: ", "").strip()

            print(f"🔎 Bulunan: {baslik}")
            context_list.append(doc.page_content)

        baglam = "\n###\n".join(context_list)

    print("-" * 40)

    # 2. ADIM: Prompt
    prompt = f"""Sen Türkçe konuşan bir film uzmanısın.
Aşağıdaki FİLM BİLGİLERİ'ni kullanarak soruyu yanıtla.

FİLM BİLGİLERİ:
{baglam}

SORU: {soru}

KURALLAR:
1. Sadece listedeki filmleri kullan.
2. Cevabı uzatma, kısa ve net ol.
3. Cevabı şu formatta ver: "Önerim: [Film Adı]. Konusu: [Özet]"

CEVAP:"""

    # 3. ADIM: Üretim
    # return_full_text=False parametresi prompt'un tekrar yazılmasını engeller
    sonuc = llm_pipeline(prompt, return_full_text=False)[0]["generated_text"]

    # Temizlik (Tekrarlayan cevapları silmek için)
    cevap = sonuc.strip()
    if "Önerim:" in cevap:
        # Bazen model cevabı iki kere yazar, ilkini alalım.
        cevap = "Önerim:" + cevap.split("Önerim:")[1]
        # Eğer arkasından tekrar "Önerim:" geliyorsa keselim
        if "Önerim:" in cevap[1:]:
             cevap = cevap.split("Önerim:")[0]

    return cevap

In [ ]:
print("\n================================================")
print("🎬 NETFLIX GURUSU (Çıkmak için 'çıkış' yazın)")
print("================================================\n")

while True:
    soru = input("Soru sor: ")
    if soru.lower() in ['çıkış', 'exit', 'quit']:
        print("İyi seyirler 👋")
        break

    print("🔎 Araştırıyorum...")
    cevap = rag_yanit(soru)
    print(f"\n🤖 AI: {cevap}\n")
    print("-" * 40)
